### Backtesting Momentum BB Strat on S&P 500

trading rules:
- Above 300 day moving average
- Stock closes above the upper bollinger band (200-period, 4 stddev)
- Exit when stock closes below the lower bollinger band (200-period, .5 stddev)
- 10% capital position size
- Daily candlesticks (1 candestick = 24 day)

In [1]:
import pandas as pd
#getting the s&p list tickers
symbols = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
symbols = symbols.Symbol.to_list()
symbols[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP']

In [ ]:
import yfinance as yf
#downloading each dataframe
dfs = {}
for symbol in symbols:
    symbol = symbol.replace(".","-") #yf can't find stock tickers like BRK.B
    dfd = yf.download(symbol, start='2016-01-01', end='2021-12-31') 
    dfs[symbol] = pd.DataFrame(dfd) #array of dataframes

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CEG: Data doesn't exist for startDate = 1451577600, endDate = 1640880000
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*******

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
import ta

def applyindicators(df):
    df["SMA_200"] = df.Close.rolling(200).mean() #200 day moving average
    df['SMA_20'] = df.Close.rolling(20).mean() #20 day moving average
    df['stddev'] = df.Close.rolling(20).std() #20 day standard deviation
    df["upper"] = df.SMA_20 + 2.5 * df.stddev #upper bolinger band
    df["lower"] = df.SMA_20 - 2.5 * df.stddev #lower bolinger band
    df["rsi"] = ta.momentum.rsi(df.Close, 10)

In [ ]:
import numpy as np

def conditions(df):
    #buy if closing price under lower BB & above 200 day MA & is 3% under the days low
    df["Buy"] = np.where((df.Close < df.lower) &
                        (df.Close > df.SMA_200) &
                        
                         (df.rsi < 30), 1, 0)
    df["Sell"] = np.where((df.rsi > 70), 1, 0)
    df["Buyprice"] = df.Close
    df["Sellprice"] = df.Close.shift(-1)

In [ ]:
def matchtrades(df):
    Buy_Sells = df[(df.Buy == 1) | (df.Sell == 1)]
    matched_Buy_Sells = Buy_Sells[(Buy_Sells.Buy.diff() == 1) | (Buy_Sells.Sell.diff() == 1)]
    return matched_Buy_Sells

In [ ]:
trades = {}

for symbol in symbols:
    symbol = symbol.replace(".","-") #yf can't find stock tickers like BRK.B
    dfe = dfs[symbol]
    dfe.Close = dfe["Adj Close"]
    applyindicators(dfe)
    conditions(dfe)
    trade = matchtrades(dfe)
    if len(trade) > 0:
        trades[symbol] = trade
        
tradesdf = pd.concat(trades).reset_index().set_index("Date")
tradesdf 

In [ ]:
import datetime as dt

In [ ]:
tradesdf["profit"] = (tradesdf.Sellprice.shift(-1) - tradesdf.Buyprice) / tradesdf.Buyprice

if len(tradesdf)%2 != 0:
    tradesdf = tradesdf[:-1:]
    
profitsdf = pd.DataFrame({"ticker": tradesdf[::2].level_0,
                         "profit": tradesdf[::2].profit,
                         "Buydates": tradesdf[::2].index,
                         "Selldates": tradesdf[1::2].index}).reset_index(drop=True)

profitsdf["time_in_market"] = profitsdf.Selldates - profitsdf.Buydates
profitsdf = profitsdf.drop_duplicates(subset='Buydates', keep='first')
sorted_profits = profitsdf.sort_values("Buydates")
# sorted_profits = sorted_profits[sorted_profits["time_in_market"] < pd.Timedelta(11, "D")]
sorted_profits = sorted_profits.reset_index(drop=True)

lst = []
for x in range(len(sorted_profits)-1):
    if sorted_profits.Selldates.iloc[x] > sorted_profits.Buydates.iloc[x+1]:
        lst.append(x)
sorted_profits.drop(lst, inplace=True)
#maximum time in market is 10 days, you usually lose when it's more than 10 days

sorted_profits = sorted_profits.reset_index(drop=True)
sorted_profits

In [ ]:
sorted_profits['returns'] = 100*sorted_profits.profit

temp = sorted_profits.returns[0]
tot = [temp]
for x in range(len(sorted_profits)-1):
    temp += sorted_profits.returns[x+1]
    temp = temp
    tot.append(temp)
sorted_profits['total_returns'] = tot
sorted_profits.sort_values('profit', ascending=False).head(10)

In [ ]:
sorted_profits['year'] = sorted_profits.Selldates.dt.year
sorted_profits['month'] = sorted_profits.Selldates.dt.month
cp = sorted_profits.groupby(['year', 'month']).profit.mean().reset_index()

inve = [100]
for x in range(len(cp)-1):
    if cp.year[x] == cp.year[x+1] and cp.month[x] < cp.month[x+1]:
        inve.append(((cp.month[x+1]-cp.month[x])*100)+inve[-1])
    elif cp.year[x] != cp.year[x+1]:
        inve.append(cp.month[x+1]*100+((12-cp.month[x])*100)+inve[-1])
# inv = [100*x for x in range(1, len(cp)+1)]
cp['invested'] = inve
cp['monthly_returns'] = cp.invested * cp.profit

capcom = [cp.invested[0] + cp.monthly_returns[0]]
for y in range(len(cp)-1):
    temp = capcom[-1]
    capcom.append((cp.invested[y+1] + (temp - cp.invested[y]))*cp.profit[y+1] + 
                  cp.invested[y+1] + (temp - cp.invested[y]))
cp['compound_returns'] = capcom
cp

In [ ]:
from matplotlib import pyplot as plt

timeframe = tradesdf.index.max() - tradesdf.index.min()

plt.style.use(["dark_background"])
plt.figure(figsize=(12,6))
(cp.compound_returns).plot()
plt.title(f"{timeframe.days} Day Backtest")
plt.show()

print(f"{round((len([x for x in sorted_profits.profit if x > 0])/len(sorted_profits)), 3)*100}% win rate")
print(f"{round((sorted_profits.profit.mean() * 100), 2)}% average profit every {sorted_profits.time_in_market.mean().days} days")
print(f"maximum loss of {round((sorted_profits.profit.min() * 100), 2)}% a trade")
print(f"maximum profit of {round((sorted_profits.profit.max() * 100), 2)}% a trade")